# Preprocessing workflow - All decisions undertaken are statistically supported by our eda analysis,
- `Handling missing data` - Drop rows where the 'Customer ID' column has null values. Analysis suggest it is the catalyst for other enull values in the dataset.
- `Handling outliers` - Drop rows where outlier has a value, but has the value zero (0) in columns like 'Price' or 'Revenue'. 
- `Handling duplicates` - There are no duplicates in the dataset
- `Datetime columns` - Extract features likes year, month, day, is_night, is_weekend, hour, minutes, seconds, quarter_year
- `Log transform` - Revenue, Price, Cost
- `One-hot encoding` - Gender, Customer_Segment, Marketing_channel, Category, Payment_Method
- `Frequency-encoding` - Subcateogry
- `Scaling` - Robust scaling

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
import sys

In [2]:
FILE_PATH = '../data/raw/customer_churn.csv'
df = pd.read_csv(FILE_PATH)

In [3]:
# Handling missing values
print(f'Observations before dropping rows : {df.shape}')
if 'Customer ID' in df.columns:
    df = df.dropna(subset=['Customer ID'])
    print(f'Observations after dropping rows : {df.shape}')
else:
    print(f"The column, 'Quantity' not found in the dataset")
    sys.exit()

Observations before dropping rows : (743486, 24)
Observations after dropping rows : (573570, 24)


In [4]:
before = len(df)
df = df[~((df['Quantity'] != 0) & ((df['Price'] == 0) | (df['Revenue'] == 0)))]
after = len(df)
print(f'Dropped {before - after} outlier rows')

Dropped 44 outlier rows


In [5]:
df['Signup_Date'] = pd.to_datetime(df['Signup_Date'])
df['Last_Login_Date'] = pd.to_datetime(df['Last_Login_Date'])

df['Customer_Lifetime_Date'] = (df['Last_Login_Date'] - df['Signup_Date']).dt.days
df['Customer_Lifetime_Date'] = df['Customer_Lifetime_Date'].clip(lower=0)

In [6]:
reference_date = df['Last_Login_Date'].max()
df['Days_Since_Last_Login'] = (reference_date - df['Last_Login_Date']).dt.days
df['Days_Since_Last_Login'] = df['Days_Since_Last_Login'].clip(lower=0)

In [7]:
# datetime features
def extract_datetime_features(df, col):

    df[col]  = pd.to_datetime(df[col], errors='coerce')

    df[f'Year_{col}'] = df[col].dt.year
    df[f'Month_{col}'] = df[col].dt.month
    df[f'Day_{col}'] = df[col].dt.day
    df[f'Quarter_{col}'] = df[col].dt.quarter
    df[f'Hour_{col}'] = df[col].dt.hour
    df[f'Minute_{col}'] = df[col].dt.minute
    df[f'Seconds_{col}'] = df[col].dt.second
    df[f'Is_weekend_{col}'] = (df[col].dt.dayofweek > 4).astype('int64')
    df[f'Is_night_{col}'] = (df[col].dt.hour > 17).astype('int64')


    return df

datetimes = ['InvoiceDate', 'Signup_Date', 'Last_Login_Date']
for datetime in datetimes:

    df = extract_datetime_features(df, datetime)


In [8]:
# log transformations
def log_transform(df, col):

    if col in df.columns:
        df[f'{col}_log'] = np.log1p(df[col])

    else:
        print(f'Column not present in dataset')
        sys.exit()

    return df
    
log_columns = ['Revenue', 'Price', 'Cost']
for log_col in log_columns:
    
    df = log_transform(df, log_col)

In [9]:
def encode_features(df_input, col):

    if col not in df_input.columns:
        return df_input
    
    dummies = pd.get_dummies(df_input[col],prefix=col, drop_first=True, dtype=int)

    df_output = pd.concat([df_input, dummies], axis=1)
    df_output = df_output.drop(columns=[col])
    

    return df_output

encode_columns = ['Gender','Payment_Method','Category','Customer_Segment','Marketing_Channel']

for encode_cols in encode_columns:

    df = encode_features(df, encode_cols)

In [10]:
df['sum_customer_profit'] = df.groupby('Customer ID')['Profit'].transform('sum') # total profit contributed by customer
df['avg_customer_profit'] = df.groupby('Customer ID')['Profit'].transform('mean') # avg profit contributed by customer

In [11]:
df['sum_description_revenue'] = df.groupby('Description')['Revenue'].transform('sum') #total revenue per item
df['avg_description_revenue'] = df.groupby('Description')['Revenue'].transform('mean') # avg revenue per item

In [12]:
df['sum_country_revenue'] = df.groupby('Country')['Revenue'].transform('sum') # total revenue per country
df['avg_country_revenue'] = df.groupby('Country')['Revenue'].transform('mean') # avg revenue per country
df['sum_country_profit'] = df.groupby('Country')['Profit'].transform('sum')
df['avg_country_profit'] = df.groupby('Country')['Profit'].transform('mean')

In [13]:
df['Customer_Transaction_Count'] = df.groupby('Customer ID')['Invoice'].transform('count')

In [14]:
# drop columns
cols_to_drop = ['Invoice','StockCode','Description','InvoiceDate','Customer ID','Signup_Date','Last_Login_Date','Country']
for cols in cols_to_drop:
    df = df.drop(columns=[cols])

In [16]:
freq = df['Subcategory'].value_counts(normalize=True).to_dict()
df['subcategory_freq'] = df['Subcategory'].map(freq)
df = df.drop(columns=['Subcategory'])

In [17]:
df

,Quantity,Price,Customer_Age,Cost,Discount_Applied,Promo_Applied,Delivery_Time_Days,Revenue,Profit,Churn_Flag,...,sum_customer_profit,avg_customer_profit,sum_description_revenue,avg_description_revenue,sum_country_revenue,avg_country_revenue,sum_country_profit,avg_country_profit,Customer_Transaction_Count,subcategory_freq
0,12,6.95,26.0,4.469733,0,0,5,83.40,29.763202,0,...,584.157071,7.038037,9894.05,32.439508,9698688.904,18.707339,3.910532e+06,7.542839,83,0.055734
1,12,6.75,29.0,3.282128,0,0,9,81.00,41.614466,1,...,584.157071,7.038037,9176.75,29.040348,9698688.904,18.707339,3.910532e+06,7.542839,83,0.110605
2,12,6.75,33.0,4.612600,0,0,7,81.00,25.648799,1,...,584.157071,7.038037,11607.50,52.522624,9698688.904,18.707339,3.910532e+06,7.542839,83,0.055281
3,48,2.10,79.0,1.120376,0,0,8,100.80,47.021953,1,...,584.157071,7.038037,11460.90,50.047598,9698688.904,18.707339,3.910532e+06,7.542839,83,0.055809
4,24,1.25,43.0,0.848714,0,0,8,30.00,9.630864,0,...,584.157071,7.038037,35192.10,18.789162,9698688.904,18.707339,3.910532e+06,7.542839,83,0.055115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743480,12,1.25,72.0,0.621793,0,0,1,15.00,7.538478,1,...,380.741310,6.345689,9802.20,19.923171,9698688.904,18.707339,3.910532e+06,7.542839,60,0.055192
743481,12,1.25,65.0,0.834504,0,0,8,15.00,4.985948,1,...,380.741310,6.345689,7429.48,16.436903,9698688.904,18.707339,3.910532e+06,7.542839,60,0.055281
743482,24,0.55,74.0,0.413610,0,0,5,13.20,3.273364,1,...,380.741310,6.345689,14728.18,16.061265,9698688.904,18.707339,3.910532e+06,7.542839,60,0.055192
743483,3,5.95,48.0,3.670415,0,0,7,17.85,6.838754,1,...,380.741310,6.345689,37525.80,23.050246,9698688.904,18.707339,3.910532e+06,7.542839,60,0.055849
